In [26]:
TI_file = f"predictions/CCG/predictions_TagInsert_1_Run1_Epoch9.csv"
TIL2R_file = f"predictions/CCG/predictions_TagInsertL2R_en_1_Run1_19.csv"

In [2]:
import os

# go to parent directory
os.chdir("..")


In [27]:
# open the files
TI_preds = []
TI_accs = []
TIL2R_preds = []
TIL2R_accs = []

with open(TI_file) as f:
    # get number of lines in the file
    rows = sum(1 for line in f)
with open(TI_file) as f:
    
    for i, line in enumerate(f):
        sentence = []
        preds = line.strip().split(", ")
        # remove the last comma
        preds[-1] = preds[-1][:-1]
        for j, pred in enumerate(preds):
            if pred.startswith("Sentence"):
                TI_accs.append(pred.split(": ")[1])
            elif i != rows - 1:
                word, gold, prediction, order = pred.split("|")
                sentence.append((word, gold, prediction, order))
        if len(sentence) != 0:
            TI_preds.append(sentence)

with open(TIL2R_file) as f:
    for i, line in enumerate(f):
        sentence = []
        preds = line.strip().split(", ")
        # remove the last comma
        preds[-1] = preds[-1][:-1]
        for j, pred in enumerate(preds):
            if pred.startswith("Sentence"):
                TIL2R_accs.append(pred.split(": ")[1])
            elif i != rows - 1:
                word, gold, prediction = pred.split("|")
                sentence.append((word, gold, prediction))
        if len(sentence) != 0:
            TIL2R_preds.append(sentence)
        
TI_accs = [float(acc) for acc in TI_accs]
TIL2R_accs = [float(acc) for acc in TIL2R_accs]

In [28]:
len(TI_accs), len(TIL2R_accs), len(TI_preds), len(TIL2R_preds)

(1910, 1910, 1910, 1910)

In [29]:
normalized_orders = []
normalized_indeces = []
for ti_preds, til2r_preds, ti_accs, til2r_accs in zip(TI_preds, TIL2R_preds, TI_accs, TIL2R_accs):
    if ti_accs > til2r_accs:
        sentence_len = len(ti_preds)
        for i, (ti, til2r) in enumerate(zip(ti_preds, til2r_preds)):
            ti_word, gold, ti_pred, ti_order = ti
            til2r_word, _, til2r_pred = til2r
            if ti_pred == gold and til2r_pred != gold:
                normalized_index = float(i) / sentence_len
                normalized_indeces.append(normalized_index)
                normalized_order = float(ti_order) / sentence_len
                normalized_orders.append(normalized_order)

len(normalized_orders), sum(normalized_orders) / len(normalized_orders), sum(normalized_indeces) / len(normalized_indeces)

(806, 0.8270463941985158, 0.49267207081379893)

In [30]:
normalized_orders = []
normalized_indeces = []
for ti_preds, til2r_preds, ti_accs, til2r_accs in zip(TI_preds, TIL2R_preds, TI_accs, TIL2R_accs):
    if ti_accs < til2r_accs:
        sentence_len = len(ti_preds)
        for i, (ti, til2r) in enumerate(zip(ti_preds, til2r_preds)):
            ti_word, gold, ti_pred, ti_order = ti
            til2r_word, _, til2r_pred = til2r
            if til2r_pred == gold and ti_pred != gold:
                normalized_index = float(i) / sentence_len
                normalized_indeces.append(normalized_index)
                normalized_order = float(ti_order) / sentence_len
                normalized_orders.append(normalized_order)

len(normalized_orders), sum(normalized_orders) / len(normalized_orders), sum(normalized_indeces) / len(normalized_indeces)

(633, 0.9286839210157678, 0.47513360471537947)

In [32]:
import json, torch

tgt_to_idx = json.load(open("data/CCG/processed/10%/CCG_to_idx.json"))
idx_to_tgt = json.load(open("data/CCG/processed/10%/idx_to_CCG.json"))
word_to_idx = json.load(open("data/CCG/processed/10%/word_to_idx.json"))
idx_to_word = json.load(open("data/CCG/processed/10%/idx_to_word.json"))
train_path = "data/CCG/processed/100%/train_data.pth"
val_path = "data/CCG/processed/100%/val_data.pth"
test_path = "data/CCG/processed/100%/test_data.pth"
train_data = torch.load(train_path)
val_data = torch.load(val_path)
test_data = torch.load(test_path)

C:\Users\bais_\AppData\Local\Temp\ipykernel_31056\1701371993.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(train_path)
C:\Users\bais_\AppData\

In [33]:
from collections import Counter

word_freqs = Counter()
tag_freqs = Counter()

for sentence in train_data['words']:
    for word in sentence:
        if word != word_to_idx['<PAD>']:
            word_string = idx_to_word[str(word)]
            word_freqs[word_string] += 1

for sentence in train_data['tags']:
    for tag in sentence:
        if tag != tgt_to_idx['<PAD>'] and tag != tgt_to_idx['<START>']:
            tag_string = idx_to_tgt[str(tag)]
            tag_freqs[tag_string] += 1

word_freqs.most_common(10), tag_freqs.most_common(10)

([(',', 52749),
  ('the', 44759),
  ('.', 42360),
  ('of', 24837),
  ('to', 24046),
  ('a', 20848),
  ('and', 17264),
  ('in', 16437),
  ("'s", 10145),
  ('that', 8650)],
 [('N', 182074),
  ('N/N', 151657),
  ('NP/N', 83736),
  ('PP/NP', 56438),
  (',', 52749),
  ('.', 42849),
  ('N/PP', 37774),
  ('conj', 21975),
  ('(N/N)/(N/N)', 20117),
  ('NP', 19969)])

In [34]:
normalized_orders = []
normalized_indeces = []
ti_unseen_words_accs = []
til2r_unseen_words_accs = []
for ti_preds, til2r_preds, ti_accs, til2r_accs in zip(TI_preds, TIL2R_preds, TI_accs, TIL2R_accs):
    # calculate accuracy of the two models on unseen words
    for i, (ti, til2r) in enumerate(zip(ti_preds, til2r_preds)):
        ti_word, gold, ti_pred, ti_order = ti
        til2r_word, _, til2r_pred = til2r
        if ti_word not in word_freqs:
            ti_unseen_words_accs.append(ti_pred == gold)
            normalized_order = float(ti_order) / len(ti_preds)
            normalized_orders.append(normalized_order)
            til2r_unseen_words_accs.append(til2r_pred == gold)
            normalized_index = float(i) / len(ti_preds)
            normalized_indeces.append(normalized_index)

print(f"Number of unseen words: {len(ti_unseen_words_accs)}")
print(f"TI accuracy on unseen words: {sum(ti_unseen_words_accs) / len(ti_unseen_words_accs)}")
print(f"TIL2R accuracy on unseen words: {sum(til2r_unseen_words_accs) / len(til2r_unseen_words_accs)}")
print(f"Average normalized order of unseen words: {sum(normalized_orders) / len(normalized_orders)}")
print(f"Average normalized index of unseen words: {sum(normalized_indeces) / len(normalized_indeces)}")


Number of unseen words: 1618
TI accuracy on unseen words: 0.9276885043263288
TIL2R accuracy on unseen words: 0.9252163164400494
Average normalized order of unseen words: 0.5993527893477102
Average normalized index of unseen words: 0.45067907892956977


In [35]:
frequent_tags = [tag for tag, freq in tag_freqs.items() if freq >= 100]
common_tags = [tag for tag, freq in tag_freqs.items() if freq >= 10 and freq < 100]
rare_tags = [tag for tag, freq in tag_freqs.items() if freq < 10]

len(frequent_tags), len(common_tags), len(rare_tags)

(203, 321, 1089)

In [36]:
normalized_freq_orders, normalized_freq_indeces = [], []
normalized_common_orders, normalized_common_indeces = [], []
normalized_rare_orders, normalized_rare_indeces = [], []
normalized_oov_orders, normalized_oov_indeces = [], []
ti_freq_tags_accs, til2r_freq_tags_accs = [], []
ti_common_tags_accs, til2r_common_tags_accs = [], []
ti_rare_tags_accs, til2r_rare_tags_accs = [], []
ti_oov_tags_accs, til2r_oov_tags_accs = [], []
for ti_preds, til2r_preds, ti_accs, til2r_accs in zip(TI_preds, TIL2R_preds, TI_accs, TIL2R_accs):
    for i, (ti, til2r) in enumerate(zip(ti_preds, til2r_preds)):
        ti_word, gold, ti_pred, ti_order = ti
        til2r_word, _, til2r_pred = til2r
        if gold not in frequent_tags and gold not in common_tags and gold not in rare_tags:
            ti_oov_tags_accs.append(ti_pred == gold)
            normalized_order = float(ti_order) / len(ti_preds)
            normalized_oov_orders.append(normalized_order)
            til2r_oov_tags_accs.append(til2r_pred == gold)
            normalized_index = float(i) / len(ti_preds)
            normalized_oov_indeces.append(normalized_index)
        elif gold in frequent_tags:
            ti_freq_tags_accs.append(ti_pred == gold)
            normalized_order = float(ti_order) / len(ti_preds)
            normalized_freq_orders.append(normalized_order)
            til2r_freq_tags_accs.append(til2r_pred == gold)
            normalized_index = float(i) / len(ti_preds)
            normalized_freq_indeces.append(normalized_index)
        elif gold in common_tags:
            ti_common_tags_accs.append(ti_pred == gold)
            normalized_order = float(ti_order) / len(ti_preds)
            normalized_common_orders.append(normalized_order)
            til2r_common_tags_accs.append(til2r_pred == gold)
            normalized_index = float(i) / len(ti_preds)
            normalized_common_indeces.append(normalized_index)
        elif gold in rare_tags:
            ti_rare_tags_accs.append(ti_pred == gold)
            normalized_order = float(ti_order) / len(ti_preds)
            normalized_rare_orders.append(normalized_order)
            til2r_rare_tags_accs.append(til2r_pred == gold)
            normalized_index = float(i) / len(ti_preds)
            normalized_rare_indeces.append(normalized_index)

print(f"Number of frequent tags: {len(ti_freq_tags_accs)}")
print(f"TI accuracy on frequent tags: {sum(ti_freq_tags_accs) / len(ti_freq_tags_accs)}")
print(f"TIL2R accuracy on frequent tags: {sum(til2r_freq_tags_accs) / len(til2r_freq_tags_accs)}")
print(f"Average normalized order of frequent tags: {sum(normalized_freq_orders) / len(normalized_freq_orders)}")
print(f"Average normalized index of frequent tags: {sum(normalized_freq_indeces) / len(normalized_freq_indeces)}")
print(f"Number of common tags: {len(ti_common_tags_accs)}")
print(f"TI accuracy on common tags: {sum(ti_common_tags_accs) / len(ti_common_tags_accs)}")
print(f"TIL2R accuracy on common tags: {sum(til2r_common_tags_accs) / len(til2r_common_tags_accs)}")
print(f"Average normalized order of common tags: {sum(normalized_common_orders) / len(normalized_common_orders)}")
print(f"Average normalized index of common tags: {sum(normalized_common_indeces) / len(normalized_common_indeces)}")
print(f"Number of rare tags: {len(ti_rare_tags_accs)}")
print(f"TI accuracy on rare tags: {sum(ti_rare_tags_accs) / len(ti_rare_tags_accs)}")
print(f"TIL2R accuracy on rare tags: {sum(til2r_rare_tags_accs) / len(til2r_rare_tags_accs)}")
print(f"Average normalized order of rare tags: {sum(normalized_rare_orders) / len(normalized_rare_orders)}")
print(f"Average normalized index of rare tags: {sum(normalized_rare_indeces) / len(normalized_rare_indeces)}")
print(f"Number of OOV tags: {len(ti_oov_tags_accs)}")
print(f"TI accuracy on OOV tags: {sum(ti_oov_tags_accs) / len(ti_oov_tags_accs)}")
print(f"TIL2R accuracy on OOV tags: {sum(til2r_oov_tags_accs) / len(til2r_oov_tags_accs)}")
print(f"Average normalized order of OOV tags: {sum(normalized_oov_orders) / len(normalized_oov_orders)}")
print(f"Average normalized index of OOV tags: {sum(normalized_oov_indeces) / len(normalized_oov_indeces)}")


Number of frequent tags: 45191
TI accuracy on frequent tags: 0.9480870084751388
TIL2R accuracy on frequent tags: 0.9430638844017614
Average normalized order of frequent tags: 0.51645422689856
Average normalized index of frequent tags: 0.4800486356783123
Number of common tags: 428
TI accuracy on common tags: 0.4696261682242991
TIL2R accuracy on common tags: 0.5841121495327103
Average normalized order of common tags: 0.9028993652917978
Average normalized index of common tags: 0.4020208276279044
Number of rare tags: 92
TI accuracy on rare tags: 0.0
TIL2R accuracy on rare tags: 0.21739130434782608
Average normalized order of rare tags: 0.8781339183109241
Average normalized index of rare tags: 0.3787890212260492
Number of OOV tags: 16
TI accuracy on OOV tags: 0.0
TIL2R accuracy on OOV tags: 0.0
Average normalized order of OOV tags: 0.7617364623549031
Average normalized index of OOV tags: 0.481787555240467


In [37]:
normalized_orders = []
normalized_indeces = []
right_for_ti = []
wrong_for_til2r = []
for ti_preds, til2r_preds, ti_accs, til2r_accs in zip(TI_preds, TIL2R_preds, TI_accs, TIL2R_accs):
    sentence_len = len(ti_preds)
    for i, (ti, til2r) in enumerate(zip(ti_preds, til2r_preds)):
        ti_word, gold, ti_pred, ti_order = ti
        til2r_word, _, til2r_pred = til2r
        if ti_pred == gold and til2r_pred != gold:
            right_for_ti.append(ti_pred)
            wrong_for_til2r.append(til2r_pred)
            normalized_index = float(i) / sentence_len
            normalized_indeces.append(normalized_index)
            normalized_order = float(ti_order) / sentence_len
            normalized_orders.append(normalized_order)

right_counter = Counter(right_for_ti)
wrong_counter = Counter(wrong_for_til2r)

print(f"Number of tags where TI is right and TIL2R is wrong: {len(right_for_ti)}")
print(f"Number of unique tags where TI is right and TIL2R is wrong: {len(right_counter)}")
print(f"Average normalized order of words where TI is right and TIL2R is wrong: {sum(normalized_orders) / len(normalized_orders)}")
print(f"Average normalized index of words where TI is right and TIL2R is wrong: {sum(normalized_indeces) / len(normalized_indeces)}")
print(f"Most common TI preds where TI is right and TIL2R is wrong: {right_counter.most_common(10)}")
print(f"Most common TIL2R preds where TI is right and TIL2R is wrong: {wrong_counter.most_common(10)}")

Number of tags where TI is right and TIL2R is wrong: 1007
Number of unique tags where TI is right and TIL2R is wrong: 137
Average normalized order of words where TI is right and TIL2R is wrong: 0.814929638407935
Average normalized index of words where TI is right and TIL2R is wrong: 0.49595146289663095
Most common TI preds where TI is right and TIL2R is wrong: [('N', 136), ('N/N', 81), ('PP/NP', 77), ('((S\\NP)\\(S\\NP))/NP', 60), ('N/PP', 60), ('(S[dcl]\\NP)/NP', 50), ('(N\\N)/NP', 34), ('NP/N', 28), ('(S\\NP)\\(S\\NP)', 25), ('S[adj]\\NP', 24)]
Most common TIL2R preds where TI is right and TIL2R is wrong: [('N', 105), ('PP/NP', 70), ('N/N', 70), ('N/PP', 59), ('(N\\N)/NP', 51), ('(N/N)/(N/N)', 46), ('((S\\NP)\\(S\\NP))/NP', 38), ('((S[dcl]\\NP)/PP)/NP', 29), ('S[adj]\\NP', 21), ('N\\N', 20)]


In [38]:
# count how many times both model get wrong a comma or a period
normalized_orders = []
normalized_indeces = []
ti_wrong_comma_period = []
til2r_wrong_comma_period = []
for ti_preds, til2r_preds, ti_accs, til2r_accs in zip(TI_preds, TIL2R_preds, TI_accs, TIL2R_accs):
    sentence_len = len(ti_preds)
    for i, (ti, til2r) in enumerate(zip(ti_preds, til2r_preds)):
        ti_word, gold, ti_pred, ti_order = ti
        til2r_word, _, til2r_pred = til2r
        if gold in [',', '.']:
            if ti_pred != gold:
                ti_wrong_comma_period.append(gold)
                normalized_order = float(ti_order) / sentence_len
                normalized_orders.append(normalized_order)
            if til2r_pred != gold:
                til2r_wrong_comma_period.append(gold)
                normalized_index = float(i) / sentence_len
                normalized_indeces.append(normalized_index)

ti_wrong_comma_period_counter = Counter(ti_wrong_comma_period)
til2r_wrong_comma_period_counter = Counter(til2r_wrong_comma_period)

print(f"Number of commas and periods where TI is wrong: {len(ti_wrong_comma_period)}")
print(f"Number of commas and number of periods where TI is wrong: {ti_wrong_comma_period_counter}")
print(f"Average normalized order of commas and periods where TI is wrong: {sum(normalized_orders) / len(normalized_orders)}")
print(f"Number of commas and periods where TIL2R is wrong: {len(til2r_wrong_comma_period)}")
print(f"Number of commas and number of periods where TIL2R is wrong: {til2r_wrong_comma_period_counter}")
print(f"Average normalized index of commas and periods where TIL2R is wrong: {sum(normalized_indeces) / len(normalized_indeces)}")



Number of commas and periods where TI is wrong: 2
Number of commas and number of periods where TI is wrong: Counter({',': 1, '.': 1})
Average normalized order of commas and periods where TI is wrong: 0.7707122093023255
Number of commas and periods where TIL2R is wrong: 21
Number of commas and number of periods where TIL2R is wrong: Counter({',': 16, '.': 5})
Average normalized index of commas and periods where TIL2R is wrong: 0.6248005321719486
